In [5]:
! pip install tweepy
! pip install textblob
! pip install dataset
! pip install twitter
from textblob import TextBlob 
import twitter
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import dataset
import pyspark 
import pandas as pd
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import itertools
import collections
import re

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#My registered twitter credentials for the app
access_token = "259009016-b06F8cGLjDbgloH6Iyv134OimmCwzAqYZO4fHzge"
access_secret = "d2pPf8r3nw4XQieyha4xJE3MvAsUVX3cao7gdxUaWW9Xv"
consumer_key = "h4mTRruAF229RVdI2jdQ4TxaS"
consumer_secret = "Tfmwho9cqldXuuf11hLap5XC4q30BPXgPo2q4RO7XBh1x1EjBJ"

 
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)
 

In [3]:
#Searching for tweets relating to climate change starting from March 1st, 2020. We are filtering out retweets, since that is redundant data. 
#Please also note that the tweets that I am retreiving are english tweets only. 
#We are obtaining 100,000 tweets relating to climate change starting from March 1, 2020
search_term = "#climate+change -filter:retweets"
tweets = tweepy.Cursor(api.search, q=search_term, lang="en", since='2020-03-01').items(100000)

#Storing them in a list
Climate_Change_Tweets_Raw = [tweet.text for tweet in tweets]

In [6]:
#Cleaning the dataset and removing any links - this does not influence the sentiment and further clouds the dataset with unnecessary characters
Climate_Change_Tweets_Cleaned = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split()) for tweet in Climate_Change_Tweets_Raw] 

In [7]:
Climate_Change_Tweets_Cleaned

['oxford We are talking about Charles Moore here In my opinion one of the most ignora',
 'This is exactly the time to be talking about climate change auspol NO coal oil thorium nuclear NuclearEnergy',
 'Canada is 55 on Climate Change Performance Index still hasn t figured out all fossilfuel sectors in trouble whi',
 'How Corporate Sustainability Officers Are Helping Big Tech Lessen Climate Change Impacts SDGs Climate',
 'peter We keep trying Here s how we countered the feds climate report of 2019 Thi',
 'It s the UNPRI institutional investors They are climate obsessed Pension fund socialism',
 'As Himalayas Warm Nepal s Climate Migrants Struggle to Survive To those who say climate change is fake and cri',
 'science Links to VIDEOS about CLIMATE CHANGE and SCIENCE climate change water air CO2 greenenergy solar',
 'Covid19 and Changing the Climate Climate Change Brings the Specter of New Infectious Diseases',
 'Indomitable',
 'A Rogue National Park Is Tweeting Out Climate Change Facts in

In [8]:
#We need to make the tweets in all lower case - this is because 
# if there are two of the same words but one is in upper case and the other in lower case
# then they are read/classified as two different words
# Therefore, in the clearning process, we must make all the tweets lower case

Visulization = [tweet.lower().split() for tweet in Climate_Change_Tweets_Cleaned]

In [9]:
#Downloading stopwords to aid me in cleaning out the tweets so I can visuzlize the tweets better. This package is the Natural Language Toolkit - its a collection of common words that are not useful in data analysis 
nltk.download('stopwords')
StopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rashijammi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
count_of_words = [[word for word in tweet_words if not word in StopWords]
              for tweet_words in Visulization]

word_count_Climate_Change = list(itertools.chain(*count_of_words))

count = collections.Counter(word_count_Climate_Change)

In [11]:
#These are the 25 most common words in this dataset
count.most_common(25)

[('climate', 591),
 ('change', 411),
 ('coronavirus', 103),
 ('climatechange', 70),
 ('amp', 45),
 ('pandemic', 42),
 ('covid19', 40),
 ('us', 40),
 ('global', 39),
 ('trump', 38),
 ('crisis', 38),
 ('world', 34),
 ('covid', 34),
 ('science', 33),
 ('new', 33),
 ('19', 33),
 ('fight', 32),
 ('need', 31),
 ('people', 27),
 ('environment', 27),
 ('help', 27),
 ('globalwarming', 25),
 ('carbon', 24),
 ('back', 22),
 ('news', 22)]

In [14]:
#With the use of Textblob, i will determine the sentiment of each tweet. This concept is further defined in my write up. 
import numpy as np

# Creating my training data
Sentiment_Value_Climate_Change = [[tweet.sentiment.polarity, str(tweet)] for tweet in [TextBlob(tweet) for tweet in Climate_Change_Tweets_Cleaned]]

#Here I am assignin the lables "Positive" and "Negative" to the tweets. 
Polarity_of_Tweets = pd.DataFrame(Sentiment_Value_Climate_Change, columns=["polarity", "tweet"])
Polarity_of_Tweets['Polarity'] = np.where(Polarity_of_Tweets['polarity']>=0, 'Positive', 'Negative')

In [17]:
# I am deleting the numerical polarity from the dataset, as i do not think it is needed
del Polarity_of_Tweets['polarity']

In [18]:
Polarity_of_Tweets

,tweet,Polarity
0,oxford We are talking about Charles Moore here...,Positive
1,This is exactly the time to be talking about c...,Positive
2,Canada is 55 on Climate Change Performance Ind...,Negative
3,How Corporate Sustainability Officers Are Help...,Positive
4,peter We keep trying Here s how we countered t...,Positive
5,It s the UNPRI institutional investors They ar...,Negative
6,As Himalayas Warm Nepal s Climate Migrants Str...,Positive
7,science Links to VIDEOS about CLIMATE CHANGE a...,Positive
8,Covid19 and Changing the Climate Climate Chang...,Positive
9,Indomitable,Positive


In [25]:
#Converting the dataframe into a csv file - just for storage
Polarity_of_Tweets.to_csv('Training_Set.csv')

In [29]:
import csv

In [62]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [59]:
#Creating bag of words using Polarity_of_Tweets dataframe
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

bow_climate_change = bow_vectorizer.fit_transform(Polarity_of_Tweets['tweet'])

#Splitting into training and testing datasets
training = bow_climate_change[:710,:]
testing = bow_climate_change[710:,:]

xtrain, xtest, ytrain, ytest = train_test_split(training, Polarity_of_Tweets['Polarity'], random_state=27, test_size=0.4)


In [67]:
#I am training the model with xtrain and ytrain. I will test the training model against the testing model and see how well
#predicts the sentiments
LogisticRegression().fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
prediction = LogisticRegression().predict_proba(ytest)

NotFittedError: Call fit before prediction